In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import utilityfunctions as uf

## PCA Review

1. What does "PCA" stand for?
2. What does PCA *do*?
3. In what circumstances might you use PCA?

Today's notebook is a modification of https://charlesreid1.github.io/circe/Digit%20Classification%20-%20PCA.html

## Handwritten Digits Data Set

The data set we wil be analyzing is a well known data set for handwriting recognition from https://archive.ics.uci.edu/ml/datasets/optical+recognition+of+handwritten+digits.

__Take a look at the data__.

*The independent variables*: each data point (observation) is an 8 pixel by 8 pixel grayscale image. We flatten this so each data point is represented by a 64 floating point one dimensional array. 

*The dependent variable*: the label is the number in the image, 0 ... 9.

*Reason for using PCA*: if we can project from 64 dimensions down to some number less than 10, we can fit a regression (or other!) model more efficiently.

### Load and separate and examine the training data

In [ ]:
# Let's load the training data
train = np.array(np.genfromtxt('data/optdigits/optdigits.tra', delimiter=',', dtype=int))  
print(uf.getShapeType(train))

In [ ]:
# Let's split off the labels
def split(data, ycol):
    y = data[:, ycol]
    xfirst = data[:, 0:ycol]
    xsecond = data[:, ycol+1:data.shape[1]]
    return (np.hstack((xfirst, xsecond)), y)

In [ ]:
(x, y) = split(train, train.shape[1]-1)
print(uf.getShapeType(y))
print(uf.getShapeType(x))

In [ ]:
# Let's look at one of the observations
observation = x[3].reshape(8, 8)
print(observation)
plt.imshow(observation, cmap='gray_r', interpolation='bicubic')
plt.show()

### Normalize and center the data

In [ ]:
# What's the optional preprocessing step before doing a PCA?

In [ ]:
# What's the second, non-optional, preprocessing step for PCA?

def center(data):
    centered = 
    return centered

In [ ]:
# Let's preprocess our data! Note: if we minmaxed, then the data is not yet centered; if we zscored, the data is centered; if we didn't normalize at all, we don't know.
def preprocess(data, minmax=False, zScore=False):
    if minmax == True and zScore == True:
        print("Nope, won't do that!")
        return data
    elif minmax == True:
        data = uf.minmaxGlobal(data)
        return center(data)
    elif zScore == True:
        return uf.zScore(data)   

In [ ]:
centered = preprocess(x, zScore=True)
print(uf.getShapeType(centered))

### Fit the PCA

In [ ]:
# What's the first step in fitting a PCA?

# Let's get the covariance matrix!
def getCovariance(data):
    return 

In [ ]:
covarianceMatrix = getCovariance(centered)

# What should its dimensions be?
print(uf.getShapeType(covarianceMatrix))

In [ ]:
# Let's look at the covariance matrix
fig = plt.figure(figsize=(12,12))
sns.heatmap(pd.DataFrame(covarianceMatrix), annot=False, cmap='PuOr')
plt.show()

In [ ]:
# What are the second and third steps in fitting a PCA?

def getSortedEigens(data):

    return eigenvalsSorted, eigenvecsSorted

In [ ]:
eigenvals, eigenvecs = getSortedEigens(covarianceMatrix)
print(uf.getShapeType(eigenvals))
print(uf.getShapeType(eigenvecs))
# How is the eigenvector matrix arranged?

In [ ]:
# We can visualize the eigenvectors, sorted by eigenvalue rank, and visually identify which eigenvector components dominate in each eigenvalue. This is essentially a visualization of what information the principal component analysis has judged most important.
fig = plt.figure(figsize=(14,3))
sns.heatmap(pd.DataFrame(eigenvecs[:, 0:21].T), 
            annot=False, cmap='coolwarm',
           vmin=-0.5,vmax=0.5)

plt.ylabel("Ranked Eigenvalue")
plt.xlabel("Eigenvector Components")
plt.show()

In [ ]:
# What's the fourth step in fitting a PCA?

# Let's compute the proportional variance and the cumulative sum of the proportional variance!
def getVarianceStats(eigenvals):

    return cumulativeSum, proportionalVariances

In [ ]:
cumulativeSum, proportionalVariances = getVarianceStats(eigenvals)

### Scree and Elbow plots

These are two plots that will help you move beyond "just keep the eigenvalues that account for 90% of the variance".

In [ ]:
# Let's look at the proportional variance

plt.figure(figsize=(6, 4))

plt.bar(range(len(proportionalVariances)), proportionalVariances, alpha=0.5, align='center',
        label='Proportional variance')

plt.ylabel('Proportional variance ratio')
plt.xlabel('Ranked Principal Components')
plt.title("Scree Graph")

plt.legend(loc='best')
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(6,4))
ax1 = fig.add_subplot(111)

ax1.plot(cumulativeSum)

ax1.set_ylim([0,1.0])

ax1.set_xlabel('Number of Principal Components')
ax1.set_ylabel('Cumulative explained variance')
ax1.set_title('Elbow Plot')

plt.show()

### Projecting the data

In [ ]:
# Let's project the data into this PCA space
def project(data, eigenvectors, numberOfComponents):

    return projected

In [ ]:
# Looking at this elbow plot, how many principal components do you think we want to keep?

projected = project(centered, eigenvecs, ??)
print(uf.getShapeType(projected))

### What can we do with data in PCA space? 

* Look at it!
* Fit models!

It's kind of weird to use linear regression for this dataset. Linear regression is meant for data where the labels are quantitative ordinal. How do we know this?

Although the integers 0..9 are ordered, here they are just labels. But we don't (yet!) know any other type of model to fit. What to do? 

Let's make up some y's - let's set y to the sum of the values in each row. 

But, you say, that's just a regular linear function. 

*Indeed, say I*. But if I gave you that data and didn't tell you anything, you'd have to fit a model to find that out.

In [ ]:
def makeRegressionY(data):
    y = np.array([np.sum(data[i]) for i in range(data.shape[0])])
    return y
yp = makeRegressionY(x)
print(yp.shape)

In [ ]:
# Remember y? Let's fit a linear regression model to the projected data

def fit(projected, independent, y):
    projectedandy = np.hstack((projected, np.array([y]).T))
    print(uf.getShapeType(projectedandy))
    c = uf.fitlstsq(projectedandy, independent, projectedandy.shape[1]-1)
    return c

In [ ]:
%%time
c = fit(projected, [0,1,2,3,4,5,6,7,8,9], yp)
print(c)

In [ ]:
# Did you *look* at the data? We also have test! Let's test

In [ ]:
# Let's load the test data
test = np.array(np.genfromtxt('data/optdigits/optdigits.tes', delimiter=',', dtype=int))  
print(uf.getShapeType(test))
(xt, yt) = split(test, test.shape[1]-1)

# Make sure we preprocess the same way!
centeredt = preprocess(xt, zScore=True)
print(uf.getShapeType(centeredt))

# Why are we not calculating the PCA again?
projectedt = project(centeredt, eigenvecs, 10)
print(uf.getShapeType(projectedt))

In [ ]:
%%time
yhat = uf.predict(projectedt, [0,1,2,3,4,5,6,7,8,9], c)

In [ ]:
ypt = makeRegressionY(xt)
print(uf.rsquared(ypt, yhat))

### Comparing PCA to not PCA

In [ ]:
# Let's compare with a regression fit on the original data
# We will, but don't have to, normalize and center first
centered = preprocess(x, zScore=True)
print(uf.getShapeType(centered))

In [ ]:
%%time
c1 = fit(centered, [x for x in range(centered.shape[1]-1)], yp)

In [ ]:
centeredt = preprocess(xt, zScore=True)
print(uf.getShapeType(centeredt))

In [ ]:
%%time
yhat1 = uf.predict(centeredt, [x for x in range(centeredt.shape[1]-1)], c1)

In [ ]:
print(uf.rsquared(ypt, yhat1))

## Resources

* https://www.displayr.com/8-tips-for-interpreting-r-squared/